In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor
import scipy.ndimage as sn

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull
from skimage import measure
import warnings

/tmp/ipykernel_7242/1963948903.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import plotlib

In [3]:
#name_project = 'project/output_FJSJ_17-01/'               
#name_project = 'project_repartition_v3.0/output_repar_v9.1_02-01/'     
name_project = 'project_repartition_v4.0/output_repar_v9.2_01-01/'             
name_project

'project_repartition_v4.0/output_repar_v9.2_01-01/'

In [4]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  ./
dir_project_workspace:  ./
dir_project:  ./project_repartition_v4.0/output_repar_v9.2_01-01/


In [5]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [6]:
key_subworks = info_basic['key_subworks']
key_subworks = info_basic['key_subworks_repick']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03',
 '19--27-03',
 '20--28-03',
 '21--29-03',
 '22--30-03',
 '23--31-03',
 '24--32-03',
 '25--33-03',
 '26--34-03',
 '27--35-03',
 '28--36-03',
 '29--37-03',
 '30--41-03',
 '31--42-03',
 '32--43-03',
 '33--46-03',
 '34--47-03',
 '35--48-03',
 '36--52-03',
 '37--53-03',
 '38--07-04',
 '39--08-04',
 '40--09-04',
 '41--10-04',
 '42--12-04',
 '43--13-04',
 '44--17-04',
 '45--20-04',
 '46--21-04',
 '47--22-04',
 '48--23-04',
 '49--24-04',
 '50--25-04',
 '51--27-04',
 '52--28-04',
 '53--29-04',
 '54--30-04',
 '55--31-04',
 '56--32-04',
 '57--33-04',
 '58--34-04',
 '59--35-04',
 '60--36-04',
 '61--41-04',
 '62--42-04',
 '63--47-04',
 '64--48-04',
 '65--49-04',
 '66--52-04',
 '67--53-04',
 '68--08-05',
 '69--12-05',
 '70--13-05',
 '71--14-05',
 '72--16-05',
 

In [7]:
dir_image = dir_project+info_basic['dir_image'] + 'autopick/'
if not os.path.exists(dir_image):
    os.makedirs(dir_image)
dir_ds = dir_project+info_basic['dir_ds']
print('dir_image: ', dir_image)
print('dir_ds: ', dir_ds)

dir_image:  ./project_repartition_v4.0/output_repar_v9.2_01-01/image_v9.2_01-01/autopick/
dir_ds:  ./project_repartition_v4.0/output_repar_v9.2_01-01/ds_v9.2_01-01/


In [8]:
dir_disp =  'autopick'+info_basic['tag']+'/'
info_basic['dir_disp'] = dir_disp
dir_disp = dir_project+dir_disp
if not os.path.exists(dir_disp):
    os.makedirs(dir_disp)
print('dir_disp: ', dir_disp)

dir_disp:  ./project_repartition_v4.0/output_repar_v9.2_01-01/autopickv9.2_01-01/


In [9]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)

In [10]:
ds_remove_all = {}
for key_subwork in key_subworks:
    print(key_subwork)
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]
    ds.close()
    ds_remove_all[key_subwork] = ds_remove

1--08-03
2--09-03
3--10-03
4--11-03
5--12-03
6--13-03
7--14-03
8--15-03
9--16-03
10--18-03
11--19-03
12--20-03
13--21-03
14--22-03
15--23-03
16--24-03
17--25-03
18--26-03
19--27-03
20--28-03
21--29-03
22--30-03
23--31-03
24--32-03
25--33-03
26--34-03
27--35-03
28--36-03
29--37-03
30--41-03
31--42-03
32--43-03
33--46-03
34--47-03
35--48-03
36--52-03
37--53-03
38--07-04
39--08-04
40--09-04
41--10-04
42--12-04
43--13-04
44--17-04
45--20-04
46--21-04
47--22-04
48--23-04
49--24-04
50--25-04
51--27-04
52--28-04
53--29-04
54--30-04
55--31-04
56--32-04
57--33-04
58--34-04
59--35-04
60--36-04
61--41-04
62--42-04
63--47-04
64--48-04
65--49-04
66--52-04
67--53-04
68--08-05
69--12-05
70--13-05
71--14-05
72--16-05
73--17-05
74--19-05
75--22-05
76--24-05
77--26-05
78--27-05
79--28-05
80--29-05
81--30-05
82--32-05
83--33-05
84--35-05
85--36-05
86--37-05
87--38-05
88--39-05
89--40-05
90--41-05
91--42-05
92--43-05
93--44-05
94--48-05
95--49-05
96--51-05
97--52-05
98--53-05
99--54-05
100--07-06
101--08-

### Pick probes' Phase velocity automatically

In [11]:
def cluster_fundamental(ds):
    global f
    global c
    global f_new
    global c_new
    global dir_ds
    global threshold
    # 归一化
    #ds = ds / np.max(ds)  
    ds = np.nan_to_num(ds)
    for i in range(ds.shape[1]):
        ds[:,i] = ds[:,i]/np.max(ds[:,i])

    binary_ds = ds > threshold
    label_ds,num_labels = measure.label(binary_ds, connectivity=2, return_num=True)
    region_areas = [np.sum(label_ds==i) for i in range(1,num_labels+1)]
    if num_labels == 0:
        return np.zeros(0),np.zeros(0)
    max_area_label = np.argmax(region_areas) + 1
    largest_cluster = label_ds == max_area_label
    indices = np.argwhere(largest_cluster)

    ff = f_new[indices[:,1]]
    cc = c_new[indices[:,0]]

    return ff,cc

In [12]:
def pick_fundamental(key):
    global f
    global c
    global f_new
    global c_new
    global flag_f
    global flag_c
    global dir_ds
    global threshold
    global dir_disp
    global dir_ds
    global info_basic
    global info_basic_bi
    global smooth_cluster
    global smooth_pick
    global key_subworks
    global ds_remove_all

    
    ds_remove = ds_remove_all[key]
    grace = 0.0

    f_start = flag_f[0]
    f_end = flag_f[1]
    c_min = flag_c[0]
    c_max = flag_c[1]

    ff_all = []
    cc_all = []


    while f_start != f_end:
        #print(f_start,f_end)
        f_new = f[np.logical_and(f>f_start,f <= f_end)]
        c_new = c[np.logical_and(c>c_min,c < c_max)]
        ds_new0 = ds_remove[np.logical_and(c>c_min,c < c_max),:]
        ds_new = ds_new0[:,np.logical_and(f>f_start,f <= f_end)]
        # 对数组进行横向的平滑
        ds_new = pd.DataFrame(ds_new)
        #ds_new = ds_new.rolling(100).mean()
        ds_new = ds_new.T
        ds_new = ds_new.rolling(smooth_cluster).mean()
        ds_new = np.array(ds_new)
        ds_new = ds_new.T
        ff,cc = cluster_fundamental(ds_new)
        if len(ff_all) == 0:
            ff_all = ff
            cc_all = cc
        elif len(ff) != 0:
            ff_all = np.hstack((ff_all,ff))
            cc_all = np.hstack((cc_all,cc))
        else:
            #print('this')
            break
        f_start = np.max(ff)
        c_max = np.max(cc[ff==f_start]) + grace
    
    # smooth
    # 转置
    """
    ds_smooth = ds_remove.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    ds_smooth = ds_smooth.T
    ds_smooth = pd.DataFrame(ds_smooth)
    ds_smooth = ds_smooth.rolling(smooth_pick).mean()
    ds_smooth = np.array(ds_smooth)
    """
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = smooth_pick,level_c = smooth_pick)

    f_range = set(ff_all)
    f_range = np.array(list(f_range))
    c_pick = []
    for i in range(len(f_range)):
        f_this = f_range[i]
        c_this = cc_all[ff_all==f_this]
        c_range = [np.min(c_this),np.max(c_this)]
        c_new = c[np.logical_and(c>=c_range[0],c<=c_range[1])]
        c_pick.append(c_new[np.argmax(ds_smooth[np.logical_and(c>=c_range[0],c<=c_range[1]),f==f_this])])
    c_pick = np.array(c_pick)
    # smooth
    index = np.argsort(f_range)
    f_range = f_range[index]
    c_pick = c_pick[index]
    c_pick_smooth = sn.gaussian_filter1d(c_pick,6)
    
    # save as txt
    np.savetxt(dir_disp+'autopick_cluster_'+key+'.txt', np.array([ff_all,cc_all]).T, fmt='%.4f', delimiter=',')
    np.savetxt(dir_disp+'autopick_'+key+'.txt', np.array([f_range,c_pick,c_pick_smooth]).T, fmt='%.4f', delimiter=',')
    print('finish '+ key + ' '+str(key_subworks.index(key)+1) + '/'+str(len(key_subworks)))

In [13]:
f = info_basic_bi['f']
c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
flag_c = [0.25,1.2]
flag_f = [3,15]
#flag_f = [6,25]
threshold = 0.9
f_new = f[np.logical_and(f>flag_f[0],f < flag_f[1])]
c_new = c[np.logical_and(c>flag_c[0],c < flag_c[1])]
smooth_cluster = 30
smooth_pick = 3
flag_pick = 1

In [14]:
warnings.filterwarnings("ignore")
if flag_pick == 1:
    for key_subwork in key_subworks:
        pick_fundamental(key_subwork)

finish 1--08-03 1/597
finish 2--09-03 2/597
finish 3--10-03 3/597
finish 4--11-03 4/597
finish 5--12-03 5/597
finish 6--13-03 6/597
finish 7--14-03 7/597
finish 8--15-03 8/597
finish 9--16-03 9/597
finish 10--18-03 10/597
finish 11--19-03 11/597
finish 12--20-03 12/597
finish 13--21-03 13/597
finish 14--22-03 14/597
finish 15--23-03 15/597
finish 16--24-03 16/597
finish 17--25-03 17/597
finish 18--26-03 18/597
finish 19--27-03 19/597
finish 20--28-03 20/597
finish 21--29-03 21/597
finish 22--30-03 22/597
finish 23--31-03 23/597
finish 24--32-03 24/597
finish 25--33-03 25/597
finish 26--34-03 26/597
finish 27--35-03 27/597
finish 28--36-03 28/597
finish 29--37-03 29/597
finish 30--41-03 30/597
finish 31--42-03 31/597
finish 32--43-03 32/597
finish 33--46-03 33/597
finish 34--47-03 34/597
finish 35--48-03 35/597
finish 36--52-03 36/597
finish 37--53-03 37/597
finish 38--07-04 38/597
finish 39--08-04 39/597
finish 40--09-04 40/597
finish 41--10-04 41/597
finish 42--12-04 42/597
finish 43-

In [15]:
def plot_disp(key_subwork):
    global dir_disp
    global f
    global c
    global dir_image
    global key_subworks

    fig,ax = plt.subplots(1,3,figsize=(18,6))
    dir_ds = dir_project + info_basic['dir_ds']
    ds = h5py.File(dir_ds+'ds_'+key_subwork+'.h5', 'r')
    ds_remove = ds['ds_remove'][0]

    title0 = key_subwork
    ff_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,0]
    cc_all = np.loadtxt(dir_disp+'autopick_cluster_'+key_subwork+'.txt',delimiter=',')[:,1]
    f_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,0]
    c_pick = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,1]
    c_smooth = np.loadtxt(dir_disp+'autopick_'+key_subwork+'.txt',delimiter=',')[:,2]

    ax[0] = plotlib.plot_fj(ax[0],ds_remove,title0,f,c,0,c_map='jet')
    ax[0].scatter(ff_all,cc_all,marker='o',s=10,color='k')
    
    ds_smooth = plotlib.smooth_ds(ds_remove,level_f = 10,level_c=6)


    ax[1] = plotlib.plot_fj(ax[1],ds_smooth,title0,f,c,0,c_map='jet')
    ax[1].scatter(f_pick,c_pick,marker='o',s=10,color='k')

    ax[2] = plotlib.plot_fj(ax[2],ds_smooth,title0,f,c,0,c_map='jet')
    ax[2].scatter(f_pick,c_smooth,marker='o',s=10,color='k')

    plt.savefig(dir_image+'autopick_'+key_subwork+'.png',dpi=60)
    plt.tight_layout()
    plt.close()
    print('finish '+key_subwork,' ', str(key_subworks.index(key_subwork)+1),'/',str(len(key_subworks)))

In [16]:
flag_plot = 1

In [17]:
if flag_plot == 1:
    for key_subwork in key_subworks:
        plot_disp(key_subwork)

finish 1--08-03   1 / 597
finish 2--09-03   2 / 597
finish 3--10-03   3 / 597
finish 4--11-03   4 / 597
finish 5--12-03   5 / 597
finish 6--13-03   6 / 597
finish 7--14-03   7 / 597
finish 8--15-03   8 / 597
finish 9--16-03   9 / 597
finish 10--18-03   10 / 597
finish 11--19-03   11 / 597
finish 12--20-03   12 / 597
finish 13--21-03   13 / 597
finish 14--22-03   14 / 597
finish 15--23-03   15 / 597
finish 16--24-03   16 / 597
finish 17--25-03   17 / 597
finish 18--26-03   18 / 597
finish 19--27-03   19 / 597
finish 20--28-03   20 / 597
finish 21--29-03   21 / 597
finish 22--30-03   22 / 597
finish 23--31-03   23 / 597
finish 24--32-03   24 / 597
finish 25--33-03   25 / 597
finish 26--34-03   26 / 597
finish 27--35-03   27 / 597
finish 28--36-03   28 / 597
finish 29--37-03   29 / 597
finish 30--41-03   30 / 597
finish 31--42-03   31 / 597
finish 32--43-03   32 / 597
finish 33--46-03   33 / 597
finish 34--47-03   34 / 597
finish 35--48-03   35 / 597
finish 36--52-03   36 / 597
finish 37-